#Foundation


In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
def calcWF(row):
  countci, countfs, countcr, countgr = 0, 0, 0, 0
  text = str(row.Title) + str(row.Description) + str(row.Text)
  text = text.lower()

  for word in ciwords:
    res = len(re.findall(word, text))
    countci += res
  for word in fswords:
    res = len(re.findall(word, text))
    countfs += res
  for word in crwords:
    res = len(re.findall(word, text))
    countcr += res
  for word in grwords:
    res = len(re.findall(word, text))
    countgr += res

  row['ci'] = countci
  row['fs'] = countfs
  row['cr'] = countcr
  row['gr'] = countgr
  return row

In [6]:
ciwords = ["dressing up", "social media", "movement", "culture", "cultural", "trending"]
fswords = ["box office", "opening weekend", "earned", "earning", "earns", "$", "million", "grossing"]
crwords = ["rated", "ratings", "critical","critic", "critics", "acclaim", "acclaimed", "review", "reviewer", "reviews", "rotten tomatoes", "imdb" ]
grwords = ["release date", "comes out", "coming out", "directed by", "starring"]

In [60]:
def getLabel(row):
  ci = row.ci
  fs = row.fs
  cr = row.cr
  gr = row.gr
  if ci>fs and ci>cr and ci>gr:
    row["label"] = "ci"
  elif fs>ci and fs>cr and fs>gr:
    row["label"] = "fs"
  elif cr>ci and cr>fs and cr>gr:
    row["label"] = "cr"
  elif gr>ci and gr>fs and gr>cr:
    row["label"] = "gr"

  return row

#Category decider


In [ ]:
movielist = ["oppenheimer_news.csv", "barbie_news.csv", "bluebeetle_news.csv", "indianajones_news.csv", "tmnt_news.csv", "missionimpossible_news.csv"]

dflist = []
for movie in movielist:
  label = movie.split("_")[0]
  print(label)
  df = pd.read_csv(movie)
  df = df.loc[:, :'URL']

  #print(movie)
  #print("before nan:", df.shape)
  df.dropna(inplace=True)

  df["label"] = label
  dflist.append(df)
  print("after nan:", df.shape)

aggdf = pd.concat(dflist)


oppenheimer
after nan: (152, 6)
barbie
after nan: (309, 6)
bluebeetle
after nan: (309, 6)
indianajones
after nan: (295, 6)
tmnt
after nan: (395, 6)
missionimpossible
after nan: (371, 6)


In [ ]:
chosenaggdf = aggdf[['Title','Description', 'label']]

In [ ]:
chosenaggdf

,Title,Description,label
0,‘Oppenheimer’ Review: A Man for Our Time,"Christopher Nolan’s complex, vivid portrait of...",oppenheimer
1,Christopher Nolan's 'Oppenheimer' Is Terrifyin...,"Nolan's latest film, if not his best, is certa...",oppenheimer
2,Inside Christopher Nolan’s 57-day race to shoo...,"For his new film, ""Oppenheimer,"" starring Cill...",oppenheimer
3,‘Barbenheimer’ Weekend Was a Real Team Effort,The bumper box office broke the usual pattern ...,oppenheimer
4,Box Office: ‘Barbie’ Opens to Record-Setting $...,"""Barbie"" and ""Oppenheimer"" shattered expectati...",oppenheimer
...,...,...,...
560,Tom Cruise Shot ‘Mission: Impossible 7’ Motorc...,"Tom Cruise filmed the ""Dead Reckoning"" motorcy...",missionimpossible
561,"The story behind Barbenheimer, the summer’s mo...",The very online showdown between Greta Gerwig’...,missionimpossible
562,Hayley Atwell And Chris McQuarrie Reveal Why T...,We can listen to Christopher McQuarrie talk ab...,missionimpossible
563,Is This the End for Christopher Nolan: Barbie ...,"Opening projection for Barbie, Oppenheimer and...",missionimpossible


In [ ]:
movielist = ["oppenheimer_news.csv", "barbie_news.csv", "bluebeetle_news.csv", "indianajones_news.csv", "tmnt_news.csv", "missionimpossible_news.csv"]

dflist = []
for movie in movielist:
  label = movie.split("_")[0]
  print(label)
  df = pd.read_csv(movie)
  df = df.loc[:, :'URL']
  df.dropna(inplace=True)

  df["label"] = label

  if label == "barbie":
    df = df.head(200)
  else:
    df = df.head(100)
  print("after nan:", df.shape)
  dflist.append(df)

aggdfshortened = pd.concat(dflist)

oppenheimer
before nan: (152, 6)
after nan: (100, 6)
barbie
before nan: (309, 6)
after nan: (200, 6)
bluebeetle
before nan: (309, 6)
after nan: (100, 6)
indianajones
before nan: (295, 6)
after nan: (100, 6)
tmnt
before nan: (395, 6)
after nan: (100, 6)
missionimpossible
before nan: (371, 6)
after nan: (100, 6)


In [ ]:
chosenaggdfshortened = aggdfshortened[['Title','Description', 'label']]

In [ ]:
chosenaggdfshortened  = chosenaggdfshortened.sample(frac = 1)
chosenaggdfshortened.to_csv("dataForAnnotation.csv")

#Oppenheimer

In [62]:
def categorize(movietitle):
  df = pd.read_csv(movietitle)
  x = df.apply(calcWF, axis=1)
  y = x.apply(getLabel, axis=1)
  labels = list(y.label)
  d = {x:labels.count(x) for x in labels}
  return y, d

In [63]:
df, stats = categorize("oppenheimer_news.csv")

In [64]:
df

,Description,Publish Date,Text,Title,URL,ci,cr,fs,gr,label
0,"Christopher Nolan’s complex, vivid portrait of...",2023-07-19 16:16:08+00:00,The movie is based on “American Prometheus: Th...,‘Oppenheimer’ Review: A Man for Our Time,https://news.google.com/rss/articles/CBMiU2h0d...,0,1,1,1,NaN
1,"Nolan's latest film, if not his best, is certa...",2023-07-21 12:49:00+00:00,"Short of Tom Cruise, there’s been no more voca...",Christopher Nolan's 'Oppenheimer' Is Terrifyin...,https://news.google.com/rss/articles/CBMiSmh0d...,0,6,3,0,cr
2,"For his new film, ""Oppenheimer,"" starring Cill...",2023-07-19 09:00:00.094000+00:00,Style Inside Christopher Nolan’s 57-day race t...,Inside Christopher Nolan’s 57-day race to shoo...,https://news.google.com/rss/articles/CBMiV2h0d...,0,7,3,1,cr
3,The bumper box office broke the usual pattern ...,2023-07-24 20:28:59+00:00,"The incongruous duo of “Barbie,” a hot-pink co...",‘Barbenheimer’ Weekend Was a Real Team Effort,https://news.google.com/rss/articles/CBMiVmh0d...,0,0,12,0,fs
4,"""Barbie"" and ""Oppenheimer"" shattered expectati...",2023-07-23 14:40:58+00:00,“Barbenheimer” is more than just a meme. It’s ...,Box Office: ‘Barbie’ Opens to Record-Setting $...,https://news.google.com/rss/articles/CBMicmh0d...,1,8,42,4,fs
...,...,...,...,...,...,...,...,...,...,...
198,Among the heavily star-studded cast of Oppenhe...,2023-08-05 13:34:29-04:00,Paycheck is not always on the priority list fo...,Cillian Murphy’s Ungodly Weight Loss for Oppen...,https://news.google.com/rss/articles/CBMitwFod...,0,0,1,0,fs
199,"For his role as J. Robert Oppenheimer, Cillian...",2023-06-28 06:08:10-04:00,"Cillian Murphy, the man with the red right han...",“I’m not trying to do an impression”: Cillian ...,https://news.google.com/rss/articles/CBMitgFod...,0,0,1,0,fs
200,Raving about her work experience with Christop...,2023-08-10 08:05:08-04:00,Christopher Nolan’s epic biography Oppenheimer...,“Hit him. He’ll be fine”: Christopher Nolan Pu...,https://news.google.com/rss/articles/CBMijwFod...,0,2,4,0,fs
201,How does Oppenheimer compare to Nolan's other ...,2023-07-31 22:04:29+01:00,"For over 20 years, Christopher Nolan has been ...",Every Christopher Nolan Movie Ranked Worst To ...,https://news.google.com/rss/articles/CBMiT2h0d...,0,0,1,0,fs


In [65]:
stats

{nan: 46, 'cr': 54, 'fs': 98, 'ci': 3, 'gr': 2}

#Dep

In [18]:
# Python 3
import http.client, urllib.parse

conn = http.client.HTTPConnection('api.mediastack.com')

params = urllib.parse.urlencode({
    'access_key': '4583a833816d0ecb1374181b30a1cbc9',
    'categories': 'general,entertainment',
    'sort': 'published_desc',
    'keywords': 'Oppenheimer Christopher Nolan',
    'languages': 'en',
    'limit': 100,
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data = res.read()

print(data.decode('utf-8'))

{"pagination":{"limit":100,"offset":0,"count":100,"total":5824},"data":[{"author":"COURTNEY OAKES","title":"Fall Sports All-Aurora 2023: 5A runner-up Cherokee Trail puts trio on All-Aurora Boys Golf Team","description":"Sophomores Brayden Forte and Dalton Sisneros and junior Christopher O\u0027Donnell of Cherokee Trail give the Cougars \u2014 who tied for second at the Class 5A state tournament \u2014 a heavy presence on the 2023 Aurora Sentinel All-Aurora Boys Golf Team.","url":"https:\/\/sentinelcolorado.com\/sports\/preps\/boys-golf\/fall-sports-all-aurora-2023-5a-runner-up-cherokee-trail-puts-trio-on-all-aurora-boys-golf-team\/","source":"aurorasentinel","image":null,"category":"general","language":"en","country":"us","published_at":"2023-11-29T21:25:41+00:00"},{"author":"The Associated Press","title":"Texas man sentenced for threatening Georgia election officials","description":"Chad Christopher Stark, who pleaded guilty to one count of a sending a threat using a telecommunication

In [19]:
data = (data.decode('utf-8'))

In [20]:
import json
readable = json.loads(data)

In [21]:
readable

{'pagination': {'limit': 100, 'offset': 0, 'count': 100, 'total': 5824},
 'data': [{'author': 'COURTNEY OAKES',
   'title': 'Fall Sports All-Aurora 2023: 5A runner-up Cherokee Trail puts trio on All-Aurora Boys Golf Team',
   'description': "Sophomores Brayden Forte and Dalton Sisneros and junior Christopher O'Donnell of Cherokee Trail give the Cougars — who tied for second at the Class 5A state tournament — a heavy presence on the 2023 Aurora Sentinel All-Aurora Boys Golf Team.",
   'url': 'https://sentinelcolorado.com/sports/preps/boys-golf/fall-sports-all-aurora-2023-5a-runner-up-cherokee-trail-puts-trio-on-all-aurora-boys-golf-team/',
   'source': 'aurorasentinel',
   'image': None,
   'category': 'general',
   'language': 'en',
   'country': 'us',
   'published_at': '2023-11-29T21:25:41+00:00'},
  {'author': 'The Associated Press',
   'title': 'Texas man sentenced for threatening Georgia election officials',
   'description': 'Chad Christopher Stark, who pleaded guilty to one coun

In [22]:
from mediastack import MediaStack
client = MediaStack("4583a833816d0ecb1374181b30a1cbc9")

resp = client.get_live_news(categories =
 ["general", "entertainment"], languages=["en"], keywords = 'Oppenheimer', date='2023-06-21,2023-08-21' )

print(resp.pagination)
print(resp.data[0])

ModuleNotFoundError: ignored

In [ ]:
print(resp.data[4])

In [ ]:
len(resp.data)
